# PDB Trajectory Lipid and Protein Interaction Analysis

This Jupyter Notebook provides a comprehensive tool for analyzing lipid phosphorus atom distributions (heatmaps), lipid Centers of Geometry (COGs), and protein-lipid interaction frequencies from PDB trajectory files.

The script is designed for parallel processing to handle large trajectories efficiently and allows you to specify which lipid types you want to analyze.

## 1. Setup and Configuration

First, let's ensure all necessary libraries are installed and configure the analysis parameters.

In [ ]:
# Install necessary libraries if you haven't already
# Uncomment the line below and run this cell if you get ModuleNotFoundError
# !pip install numpy matplotlib seaborn

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from collections import defaultdict
import multiprocessing
import os
import csv

# --- Configuration for Analysis ---
num_heatmap_bins = 500    # Number of bins for 2D histograms (heatmaps)
interaction_cutoff = 4.5  # Angstroms, distance cutoff for protein-lipid interaction

# Parallelization settings
NUM_CORES = os.cpu_count() or 12 # Use all available CPU cores, or default to 12

# Global toggle for test mode
TEST_MODE_ENABLED = False
MAX_TEST_FRAMES = 100

# Chains considered as protein for interaction analysis (A to L)
protein_chains_to_monitor = set('ABCDEFGHIJKL')
# Residue names to exclude from protein interaction analysis (common non-protein, water, ions)
non_protein_resnames_base = {'HOH', 'WAT', 'CL', 'NA', 'MG', 'CA', 'K', 'OXY', 'ION'}

# --- Helper functions for defaultdict to make them picklable ---
def int_factory():
    return 0

def nested_int_dict_factory():
    return defaultdict(int_factory)

## 2. Worker Function for Parallel Parsing

This function will be executed by multiple processes to parse different chunks of the PDB file simultaneously. It collects P-atom coordinates, all lipid atom coordinates (for COG), CA atom coordinates (for protein trace), and protein-lipid interaction data.

In [ ]:
def parse_pdb_chunk(chunk_info):
    """
    Parses a specific range of frames from the PDB file.
    `chunk_info` is a tuple: (file_path, start_model_idx, end_model_idx, model_offsets_map, lipid_ids_to_analyze_worker, non_protein_resnames_effective_worker)
    """
    file_path, start_model_idx_global, end_model_idx_global, model_offsets_map, lipid_ids_to_analyze_worker, non_protein_resnames_effective_worker = chunk_info

    # Local data structures for this chunk/process
    local_ca_coords_model1_chains = defaultdict(list)

    # Dynamic storage for all atoms of specified lipids (for COG calculation)
    # Structure: {(LIPID_ID, model_num, chain_id, res_seq): [[x,y,z], ...]}
    local_all_lipid_residue_atoms = defaultdict(list)

    # Dynamic storage for P-atom coordinates for all specified lipids (for heatmaps)
    # Structure: {'LIPID_ID': {'x': [...], 'y': [...], 'z': [...]}}
    local_all_lipid_p_coords = defaultdict(lambda: defaultdict(list))

    # Dynamic storage for P-atom coordinates of lipids per frame (for interaction calculation)
    # Structure: {'LIPID_ID': [[x,y,z], ...]}
    local_current_frame_lipid_p_atoms = defaultdict(list)

    # Dynamic storage for protein atoms per frame (for interaction calculation)
    # Structure: {(chain_id, res_seq): [[x,y,z], ...]}
    local_current_frame_protein_atoms = defaultdict(list)

    # Dynamic storage for protein-lipid contact counts
    # Structure: {'LIPID_ID': {'CHAIN_ID': {'RES_SEQ': count}}}
    local_protein_lipid_contact_counts = defaultdict(lambda: defaultdict(nested_int_dict_factory))

    # Dynamic total frames count for interaction analysis per lipid
    # Structure: {'LIPID_ID': count}
    local_total_frames_for_lipid_interaction_analysis = defaultdict(int)

    local_current_pdb_model_num = -1
    local_frame_counter_in_chunk = 0 # Counts frames within this chunk's processing

    # Helper function for interaction calculation (local to this worker)
    def _process_interactions_for_frame_data_local():
        nonlocal local_protein_lipid_contact_counts, local_total_frames_for_lipid_interaction_analysis
        nonlocal local_current_frame_protein_atoms, local_current_frame_lipid_p_atoms

        if not local_current_frame_protein_atoms: # If no protein atoms in this frame, skip interaction check
            return

        # Prepare protein atom data for efficient distance calculation
        protein_atom_coords_np_all = []
        protein_atom_info_all = [] # To map back to chain/res_seq
        for (chain_id, res_seq), coords_list in local_current_frame_protein_atoms.items():
            for coord in coords_list:
                protein_atom_coords_np_all.append(coord)
                protein_atom_info_all.append((chain_id, res_seq))
        protein_atom_coords_np_all = np.array(protein_atom_coords_np_all)

        # Iterate through each lipid type for interaction analysis
        for lipid_id in lipid_ids_to_analyze_worker:
            if lipid_id not in local_current_frame_lipid_p_atoms or not local_current_frame_lipid_p_atoms[lipid_id]:
                continue # No P atoms for this lipid in this frame

            local_total_frames_for_lipid_interaction_analysis[lipid_id] += 1
            contacted_residues_in_this_frame_for_lipid = set()

            for lipid_p_coord in local_current_frame_lipid_p_atoms[lipid_id]:
                lipid_p_coord_np = np.array(lipid_p_coord)
                distances = np.linalg.norm(protein_atom_coords_np_all - lipid_p_coord_np, axis=1)

                contact_indices = np.where(distances <= interaction_cutoff)[0]
                for idx in contact_indices:
                    contacted_residues_in_this_frame_for_lipid.add(protein_atom_info_all[idx])

            for (chain_id, res_seq) in contacted_residues_in_this_frame_for_lipid:
                local_protein_lipid_contact_counts[lipid_id][chain_id][res_seq] += 1

    # --- Core Parsing Logic for the Chunk ---
    try:
        with open(file_path, "r") as f:
            if start_model_idx_global > 0:
                f.seek(model_offsets_map[start_model_idx_global])

            for line in f:
                if line.startswith("MODEL"):
                    model_num = int(line[5:].strip())

                    if model_num > end_model_idx_global:
                        break

                    if local_current_pdb_model_num != -1:
                        _process_interactions_for_frame_data_local()

                    local_current_pdb_model_num = model_num
                    local_frame_counter_in_chunk += 1

                    # Clear frame-specific data for the new model
                    local_current_frame_protein_atoms.clear()
                    local_current_frame_lipid_p_atoms.clear() # Clear for all lipids

                    if TEST_MODE_ENABLED and local_frame_counter_in_chunk > MAX_TEST_FRAMES:
                        break

                elif line.startswith("ATOM") and local_current_pdb_model_num != -1:
                    try:
                        atom_name = line[12:16].strip()
                        res_name = line[17:21].strip()
                        chain_id = line[21].strip()
                        res_seq = int(line[22:26].strip())
                        x_coord = float(line[30:38].strip())
                        y_coord = float(line[38:46].strip())
                        z_coord = float(line[46:54].strip())

                        # Collect all atoms for specified lipids (for COG calculation)
                        if res_name in lipid_ids_to_analyze_worker:
                            local_all_lipid_residue_atoms[(res_name, local_current_pdb_model_num, chain_id, res_seq)].append([x_coord, y_coord, z_coord])

                        # Collect CA atoms for Model 1 (protein trace)
                        if local_current_pdb_model_num == 1 and atom_name == "CA":
                            local_ca_coords_model1_chains[chain_id].append([res_seq, x_coord, y_coord, z_coord])

                        # Collect P-atom coordinates for all specified lipids (for heatmaps)
                        # We use {"P", "P1", "P2", "P3"} for broad P-atom recognition
                        if res_name in lipid_ids_to_analyze_worker and atom_name in {"P", "P1", "P2", "P3"}:
                            local_all_lipid_p_coords[res_name]['x'].append(x_coord)
                            local_all_lipid_p_coords[res_name]['y'].append(y_coord)
                            local_all_lipid_p_coords[res_name]['z'].append(z_coord)
                            # Also collect for per-frame interaction analysis if it's the specific 'P' atom
                            if atom_name == "P": # Assuming 'P' is the atom for interaction calculation
                                local_current_frame_lipid_p_atoms[res_name].append([x_coord, y_coord, z_coord])

                        # Collect protein atoms for interaction analysis
                        if chain_id in protein_chains_to_monitor and res_name not in non_protein_resnames_effective_worker:
                            local_current_frame_protein_atoms[(chain_id, res_seq)].append([x_coord, y_coord, z_coord])

                    except ValueError as e:
                        # print(f"Warning: Could not parse line (ValueError): {line.strip()} - {e}") # Uncomment for debugging parsing issues
                        pass
                    except IndexError as e:
                        # print(f"Warning: Could not parse line (IndexError): {line.strip()} - {e}") # Uncomment for debugging parsing issues
                        pass

    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found in worker {os.getpid()}.")
        return None

    # Process the last frame in the chunk
    if local_current_pdb_model_num != -1:
        _process_interactions_for_frame_data_local()

    return (
        local_all_lipid_residue_atoms,
        local_all_lipid_p_coords,
        local_ca_coords_model1_chains,
        local_protein_lipid_contact_counts,
        local_total_frames_for_lipid_interaction_analysis,
        local_frame_counter_in_chunk
    )

## 3. Main Analysis Execution

This section orchestrates the parallel parsing, aggregates results, calculates COGs, and prepares data for plotting. You will be prompted to enter the PDB file path and the lipid IDs you wish to analyze.

In [ ]:
# --- Main Script Execution ---

file_path = input("Enter the path to your PDB trajectory file (e.g., EMT_t5.pdb): ")
lipid_input_str = input("Enter lipid residue IDs to analyze, separated by space (e.g., POPG POPE PMCL): ")

lipid_ids_to_analyze = [lipid.upper() for lipid in lipid_input_str.split()]

# Dynamically update non_protein_resnames for workers
non_protein_resnames_effective = non_protein_resnames_base.copy()
for lipid_id in lipid_ids_to_analyze:
    if lipid_id in non_protein_resnames_effective:
        non_protein_resnames_effective.remove(lipid_id)


# Initialize global lists/dicts for aggregation
all_lipid_residue_atoms = defaultdict(list)
all_lipid_cogs_coords = defaultdict(lambda: defaultdict(list))

ca_coords_model1_chains = defaultdict(list)

all_lipid_p_coords = defaultdict(lambda: defaultdict(list))

protein_lipid_contact_counts = defaultdict(lambda: defaultdict(nested_int_dict_factory))
total_frames_for_lipid_interaction_analysis = defaultdict(int)

total_frames_parsed = 0


print(f"\n--- Starting Parallel PDB Parsing with {NUM_CORES} cores ---")
print(f"Analyzing PDB file: {file_path}")
print(f"Lipids to analyze: {', '.join(lipid_ids_to_analyze)}")
if TEST_MODE_ENABLED:
    print(f"TEST MODE ENABLED: Each worker will process up to {MAX_TEST_FRAMES} frames from its assigned range.")

parse_start_time = time.time()

if not os.path.exists(file_path):
    print(f"Error: File '{file_path}' not found. Please ensure it's in the correct directory.")
    # exit() # In a notebook, better to just stop execution than exit()
    raise FileNotFoundError(f"File '{file_path}' not found.")

# --- First Pass: Index MODEL records to determine byte offsets ---
print(f"First pass: Indexing MODEL records in '{file_path}'...")
model_offsets = {}
current_offset = 0
with open(file_path, "r") as f:
    for line in f:
        if line.startswith("MODEL"):
            model_num = int(line[5:].strip())
            model_offsets[model_num] = current_offset
        current_offset += len(line.encode('utf-8'))

if not model_offsets:
    print("Error: No MODEL records found in the PDB file.")
    # exit()
    raise ValueError("No MODEL records found in the PDB file.")

all_model_nums = sorted(model_offsets.keys())
num_total_frames = len(all_model_nums)
print(f"Found {num_total_frames} MODEL records (frames).")

# --- Prepare chunks for workers ---
frames_per_core = (num_total_frames + NUM_CORES - 1) // NUM_CORES

worker_chunks_info = []
for i in range(NUM_CORES):
    start_idx = i * frames_per_core
    end_idx = min(start_idx + frames_per_core, num_total_frames)

    if start_idx < num_total_frames:
        start_model_num = all_model_nums[start_idx]
        end_model_num = all_model_nums[end_idx - 1] if end_idx > start_idx else start_model_num
        worker_chunks_info.append((file_path, start_model_num, end_model_num, model_offsets, lipid_ids_to_analyze, non_protein_resnames_effective))
        print(f"Worker {i+1} will process frames from MODEL {start_model_num} to MODEL {end_model_num}")
    else:
        print(f"Worker {i+1} has no frames to process.")

with multiprocessing.Pool(NUM_CORES) as pool:
    results = pool.map(parse_pdb_chunk, worker_chunks_info)

results = [r for r in results if r is not None]

# Aggregate results from all processes
print("\n--- Aggregating results from parallel processes ---")
for (
    local_all_lipid_res_atoms,
    local_all_lipid_p_coords,
    local_ca_coords_m1_chains,
    local_protein_lipid_contact_counts,
    local_total_frames_for_lipid_interaction,
    local_frames_parsed_by_worker
) in results:
    # Aggregate all lipid residue atoms (for COG calculation)
    for key, value in local_all_lipid_res_atoms.items():
        all_lipid_residue_atoms[key].extend(value)

    # Aggregate P atom coordinates for ALL specified lipids (for heatmaps)
    for lipid_id, coords_dict in local_all_lipid_p_coords.items():
        all_lipid_p_coords[lipid_id]['x'].extend(coords_dict['x'])
        all_lipid_p_coords[lipid_id]['y'].extend(coords_dict['y'])
        all_lipid_p_coords[lipid_id]['z'].extend(coords_dict['z'])

    # Aggregate CA coords
    for chain_id, coords_list in local_ca_coords_m1_chains.items():
        ca_coords_model1_chains[chain_id].extend(coords_list)

    # Aggregate protein-lipid contact counts
    for lipid_id, chain_data in local_protein_lipid_contact_counts.items():
        for chain_id, res_data in chain_data.items():
            for res_seq, count in res_data.items():
                protein_lipid_contact_counts[lipid_id][chain_id][res_seq] += count

    # Aggregate total frames for interaction analysis
    for lipid_id, count in local_total_frames_for_lipid_interaction.items():
        total_frames_for_lipid_interaction_analysis[lipid_id] += count

    total_frames_parsed += local_frames_parsed_by_worker

parse_end_time = time.time()
print(f"\nParallel file parsing completed. Total frames processed by workers: {total_frames_parsed} in {parse_end_time - parse_start_time:.4f} seconds.")
if TEST_MODE_ENABLED:
    print(f"NOTE: Test mode was enabled, each worker processed up to {MAX_TEST_FRAMES} frames from its assigned range.")


# --- Calculate COGs for all specified lipids from aggregated atom data ---
print("\n--- Calculating Centers of Geometry for specified lipids ---")
if not all_lipid_residue_atoms:
    print("No specified lipid residues found in the PDB file. Check residue names or file content.")
else:
    for (lipid_id, model_num, chain_id, res_seq), atom_coords_list in all_lipid_residue_atoms.items():
        if atom_coords_list:
            cog = np.mean(atom_coords_list, axis=0)
            all_lipid_cogs_coords[lipid_id]['x'].append(cog[0])
            all_lipid_cogs_coords[lipid_id]['y'].append(cog[1])
            all_lipid_cogs_coords[lipid_id]['z'].append(cog[2])

for lipid_id in lipid_ids_to_analyze:
    print(f"Calculated COGs for {len(all_lipid_cogs_coords[lipid_id]['x'])} {lipid_id} residues.")


# --- Prepare CA Atom Data for Plotting ---
ca_coords_by_chain_model1_processed = {}
if not ca_coords_model1_chains:
    print("\nNo 'CA' atoms found in the first model to plot.")
else:
    for chain_id, coords_list in ca_coords_model1_chains.items():
        if coords_list:
            unique_ca_coords = {}
            for res_seq, x, y, z in coords_list:
                unique_ca_coords[res_seq] = [res_seq, x, y, z]

            ca_array = np.array(list(unique_ca_coords.values()))
            if ca_array.size > 0:
                ca_array_sorted = ca_array[ca_array[:, 0].argsort()]
                ca_coords_by_chain_model1_processed[chain_id] = ca_array_sorted
    print(f"Prepared CA data for chains: {list(ca_coords_by_chain_model1_processed.keys())} from Model 1.")


## 4. Plotting and Data Export

This section generates the various heatmaps and interaction frequency plots, and exports the raw data to CSV files in a `csv_data` directory.

In [ ]:
# --- Plotting ---

# Determine suffix for plot titles and filenames based on test mode
plot_suffix = ""
if TEST_MODE_ENABLED:
    plot_suffix = f" (First {MAX_TEST_FRAMES} Frames/Chunk)"
    filename_suffix = f"_first_{MAX_TEST_FRAMES}_frames_per_chunk"
else:
    filename_suffix = "_full_data"

# --- CSV Output Directory ---
csv_output_dir = "csv_data"
os.makedirs(csv_output_dir, exist_ok=True)
print(f"\nCSV data will be saved to: {os.path.abspath(csv_output_dir)}")


# Helper function for P-atom heatmaps and CSV export
def plot_atom_heatmap(x_data, y_data, molecule_name, atom_type, plane, bins, title_suffix, file_suffix, csv_dir):
    """Generates and saves a 2D heatmap for atom coordinates and exports data to CSV."""
    if len(x_data) > 0 and len(y_data) > 0:
        plt.figure(figsize=(10, 8))
        heatmap_data, x_edges, y_edges = np.histogram2d(x_data, y_data, bins=bins)
        plt.imshow(
            heatmap_data.T,
            extent=[x_edges.min(), x_edges.max(), y_edges.min(), y_edges.max()],
            origin='lower',
            cmap='inferno',
            aspect='auto'
        )
        cbar = plt.colorbar()
        cbar.set_label(f'Density of {molecule_name} {atom_type} Atoms')
        plt.title(f'{molecule_name} {atom_type} Atom Heatmap ({plane} Plane){title_suffix}')
        plt.xlabel(f'{plane.split("-")[0]}-coordinate')
        plt.ylabel(f'{plane.split("-")[1]}-coordinate')
        plt.grid(False)
        plt.tight_layout()
        plt.savefig(f'heatmap_{molecule_name}_{atom_type}_atom_{plane.lower()}{file_suffix}.png', dpi=300)
        plt.show()

        # Export to CSV
        csv_heatmap_filename = os.path.join(csv_dir, f'heatmap_{molecule_name}_{atom_type}_atom_{plane.lower()}_density{file_suffix}.csv')
        np.savetxt(csv_heatmap_filename, heatmap_data, delimiter=',', header='Heatmap Density Data', comments='')
        np.savetxt(os.path.join(csv_dir, f'heatmap_{molecule_name}_{atom_type}_atom_{plane.lower()}_x_edges{file_suffix}.csv'), x_edges, delimiter=',', header='X Bin Edges', comments='')
        np.savetxt(os.path.join(csv_dir, f'heatmap_{molecule_name}_{atom_type}_atom_{plane.lower()}_y_edges{file_suffix}.csv'), y_edges, delimiter=',', header='Y Bin Edges', comments='')
        print(f"Exported {molecule_name} {atom_type} Atom Heatmap ({plane}) data to CSV: {csv_heatmap_filename}")
    else:
        print(f"\nSkipping {molecule_name} {atom_type} Atom Heatmap ({plane}){title_suffix}: No data available.")

# Helper function for COG heatmaps and CSV export
def plot_cog_heatmap(x_data, y_data, molecule_name, bins, title_suffix, file_suffix, csv_dir, ca_coords_processed=None, plot_ca=False):
    """Generates and saves a 2D heatmap for COG coordinates and exports data to CSV."""
    if len(x_data) > 0 and len(y_data) > 0:
        plt.figure(figsize=(10, 8))
        heatmap_data, x_edges, y_edges = np.histogram2d(x_data, y_data, bins=bins)
        plt.imshow(
            heatmap_data.T,
            extent=[x_edges.min(), x_edges.max(), y_edges.min(), y_edges.max()],
            origin='lower',
            cmap='inferno',
            aspect='auto'
        )
        cbar = plt.colorbar()
        cbar.set_label(f'Density of {molecule_name} COGs')

        # Export to CSV (always for COG heatmaps)
        csv_heatmap_filename = os.path.join(csv_dir, f'heatmap_{molecule_name}_COG_XY_density{file_suffix}.csv')
        np.savetxt(csv_heatmap_filename, heatmap_data, delimiter=',', header=f'{molecule_name} COG Heatmap Density Data (XY)', comments='')
        np.savetxt(os.path.join(csv_dir, f'heatmap_{molecule_name}_COG_XY_x_edges{file_suffix}.csv'), x_edges, delimiter=',', header=f'{molecule_name} COG X Bin Edges (XY)', comments='')
        np.savetxt(os.path.join(csv_dir, f'heatmap_{molecule_name}_COG_XY_y_edges{file_suffix}.csv'), y_edges, delimiter=',', header=f'{molecule_name} COG Y Bin Edges (XY)', comments='')
        print(f"Exported {molecule_name} COG Heatmap (X-Y) data to CSV: {csv_heatmap_filename}")

        if plot_ca and ca_coords_processed:
            print(f"Overlaying CA trace on {molecule_name} COG heatmap...")
            for chain_id, ca_array in ca_coords_processed.items():
                csv_ca_filename = os.path.join(csv_dir, f'ca_trace_model1_chain_{chain_id}_coords{file_suffix}.csv')
                header = "Residue_Seq,X,Y,Z"
                np.savetxt(csv_ca_filename, ca_array, delimiter=',', header=header, comments='', fmt='%.4f')
                print(f"Exported CA Trace Model 1 Chain {chain_id} data to CSV: {csv_ca_filename}")

                x_ca = ca_array[:, 1]
                y_ca = ca_array[:, 2]

                if chain_id in ['C', 'D']:
                    line_color = 'yellow'
                else:
                    line_color = 'dimgrey'

                plt.plot(x_ca, y_ca,
                         color=line_color,
                         linewidth=0.8,
                         linestyle='-',
                         alpha=0.8,
                         label=f'CA Trace Chain {chain_id}')
            plt.title(f'{molecule_name} COG Heatmap in X-Y Plane with CA Trace Overlay{title_suffix}')
        elif plot_ca and not ca_coords_processed:
            plt.text(0.5, 0.5, "No CA atoms to overlay for model 1",
                     horizontalalignment='center', verticalalignment='center',
                     transform=plt.gca().transAxes, color='gray', fontsize=12)
            plt.title(f'{molecule_name} COG Heatmap in X-Y Plane{title_suffix}')
        else:
            plt.title(f'{molecule_name} COG Heatmap in X-Y Plane{title_suffix}')

        plt.xlabel('X-coordinate')
        plt.ylabel('Y-coordinate')
        plt.grid(False)
        plt.axis('equal') # Keep aspect ratio for spatial plots

        plot_name = f'heatmap_{molecule_name}_COG_XY'
        if plot_ca and ca_coords_processed:
            plot_name += '_ca_trace_overlay_transparent'
        else:
            plot_name += '_only'

        plt.savefig(f'{plot_name}{file_suffix}.png',
                    dpi=300,
                    transparent=True,
                    bbox_inches='tight',
                    pad_inches=0)
        plt.show()
    else:
        print(f"\nSkipping {molecule_name} COG Heatmap (X-Y with/without CA Trace){title_suffix}: No COG data available.")


print("\n--- Generating Plots and Exporting Data ---")


# --- Dynamic Lipid COG Heatmaps (X-Y Plane) with CA Trace Overlay (XY Plane) ---
print("\n--- Generating Lipid COG Heatmaps and CA Trace Overlays ---")
for i, lipid_id in enumerate(lipid_ids_to_analyze):
    x_cog = np.array(all_lipid_cogs_coords[lipid_id]['x'])
    y_cog = np.array(all_lipid_cogs_coords[lipid_id]['y'])

    # Only overlay CA trace for the first lipid for cleaner plots
    plot_ca_for_this_lipid = (i == 0)
    
    plot_cog_heatmap(
        x_cog, y_cog, lipid_id, num_heatmap_bins, 
        plot_suffix, filename_suffix, csv_output_dir, 
        ca_coords_by_chain_model1_processed, plot_ca_for_this_lipid
    )


# --- CA Trace (XY Plane) ONLY (Model 1) ---
# This is still useful as a standalone plot, using data exported earlier
if ca_coords_by_chain_model1_processed:
    plt.figure(figsize=(10, 8))
    ax = plt.gca()

    for chain_id, ca_array in ca_coords_by_chain_model1_processed.items():
        x_ca = ca_array[:, 1]
        y_ca = ca_array[:, 2]

        if chain_id in ['C', 'D']:
            line_color = 'yellow'
        else:
            line_color = 'dimgrey'

        plt.plot(x_ca, y_ca,
                 color=line_color,
                 linewidth=0.8,
                 linestyle='-',
                 alpha=0.8,
                 label=f'CA Trace Chain {chain_id}')

    ax.set_frame_on(False)
    ax.set_xticks([])
    ax.set_yticks([])
    plt.title(f'CA Trace in X-Y Plane (Model 1){plot_suffix}')
    plt.xlabel('')
    plt.ylabel('')
    plt.grid(False)
    plt.axis('equal')

    plt.savefig(f'plot_ca_trace_xy_only_transparent{filename_suffix}.png',
                dpi=300,
                transparent=True,
                bbox_inches='tight',
                pad_inches=0)
    plt.show()
else:
    print(f"\nSkipping CA Trace XY Only Plot{plot_suffix}: No CA atoms found in the first model to plot.")


# --- Dynamic Phosphorus Atom Heatmaps (X-Y, X-Z, and Y-Z planes) for ALL specified lipids ---
print("\n--- Generating Phosphorus Atom Heatmaps for all specified lipids ---")
for lipid_id in lipid_ids_to_analyze:
    x_data = np.array(all_lipid_p_coords[lipid_id]['x'])
    y_data = np.array(all_lipid_p_coords[lipid_id]['y'])
    z_data = np.array(all_lipid_p_coords[lipid_id]['z'])

    if x_data.size > 0 and y_data.size > 0 and z_data.size > 0:
        plot_atom_heatmap(x_data, y_data, lipid_id, "P", "X-Y", num_heatmap_bins, plot_suffix, filename_suffix, csv_output_dir)
        plot_atom_heatmap(x_data, z_data, lipid_id, "P", "X-Z", num_heatmap_bins, plot_suffix, filename_suffix, csv_output_dir)
        plot_atom_heatmap(y_data, z_data, lipid_id, "P", "Y-Z", num_heatmap_bins, plot_suffix, filename_suffix, csv_output_dir)
    else:
        print(f"\nSkipping P Atom Heatmaps for {lipid_id}: No P atom data available for this lipid.")


# --- Dynamic Protein-Lipid Interaction Frequency Plots and CSV Export ---
print(f"\n--- Generating Protein-Lipid Interaction Frequency Plots and Exporting Data{plot_suffix} ---")
total_any_lipid_interaction_data_found = any(count > 0 for count in total_frames_for_lipid_interaction_analysis.values())

if total_any_lipid_interaction_data_found and protein_lipid_contact_counts:

    for lipid_id in lipid_ids_to_analyze:
        lipid_contact_data = protein_lipid_contact_counts[lipid_id]
        frames_analyzed_for_lipid = total_frames_for_lipid_interaction_analysis[lipid_id]

        if lipid_contact_data and frames_analyzed_for_lipid > 0:
            print(f"Processing interaction data for {lipid_id}...")
            sorted_chains = sorted(lipid_contact_data.keys())

            for chain_id in sorted_chains:
                chain_data = lipid_contact_data[chain_id]
                if chain_data:
                    sorted_res_seqs = sorted(chain_data.keys())
                    frequencies = [chain_data[res_seq] / frames_analyzed_for_lipid for res_seq in sorted_res_seqs]

                    plt.figure(figsize=(12, 6))
                    plt.plot(sorted_res_seqs, frequencies, marker='o', linestyle='-', markersize=4, label=f'Chain {chain_id}')
                    plt.title(f'Contact Frequency of {lipid_id} with Protein Residues (Chain {chain_id}){plot_suffix}')
                    plt.xlabel('Residue Number')
                    plt.ylabel('Contact Frequency (Fraction of Frames in Contact)')
                    plt.grid(True, linestyle='--', alpha=0.7)
                    plt.ylim(0, 1.05)
                    plt.tight_layout()
                    plt.savefig(f'interaction_frequency_{lipid_id}_chain_{chain_id}{filename_suffix}.png', dpi=300)
                    plt.show()

                    csv_freq_filename = os.path.join(csv_output_dir, f'interaction_frequency_{lipid_id}_chain_{chain_id}{filename_suffix}.csv')
                    with open(csv_freq_filename, 'w', newline='') as csvfile:
                        writer = csv.writer(csvfile)
                        writer.writerow(['Residue_Number', 'Contact_Frequency'])
                        for res_seq, freq in zip(sorted_res_seqs, frequencies):
                            writer.writerow([res_seq, f"{freq:.6f}"])
                    print(f"Exported interaction frequency for {lipid_id} Chain {chain_id} to CSV: {csv_freq_filename}")
                else:
                    print(f"No interaction data for {lipid_id} Chain {chain_id}{plot_suffix}.")
        else:
            print(f"No interaction data available for {lipid_id}{plot_suffix}.")
else:
    print(f"\nSkipping Protein-Lipid Interaction Frequency Plots and CSV Export{plot_suffix}: No interaction data available or no frames processed for any lipid.")

print(f"\n--- All Plots Generated and Data Exported{plot_suffix} ---")
